# Post GP Estimator Robustness 
## Causal Data Generation

In [133]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats 
%matplotlib inline
from sklearn import datasets 
import seaborn as sns

from statsmodels.tsa import arima_process 

from sklearn.gaussian_process.kernels import ConstantKernel, RBF, WhiteKernel
from sklearn.gaussian_process import GaussianProcessClassifier, GaussianProcessRegressor 

__Causal Data__

In [134]:
np.random.seed(69)
#x = arima_process.arma_generate_sample(ar=[1, -.9], ma=[1],  nsample=100)  
#y = arima_process.arma_generate_sample(ar=[1, -.8], ma=[1], nsample=100)
# phi parameters are negated 
x1= np.linspace(0,10, 1000)
x2= np.sin(x1**2) 

x4 = np.random.choice(np.arange(1,50), 1000)
x5 = np.random.choice(np.arange(50,100), 1000)

x3_nn =  x5 + x4  # generation process actually depend on x5
x3_high = x3_nn + np.random.normal(0, 1, 1000) 
x3_med = x3_nn + np.random.normal(0,15, 1000)
x3_low = x3_nn + np.random.normal(0,100, 1000)

print(f"high:{stats.pearsonr(x5, x3_high)} \n med: {stats.pearsonr(x5, x3_med)} \n low: {stats.pearsonr(x5, x3_low)}")

high:(0.7199256515509557, 1.7918294802920872e-160) 
 med: (0.5591049355469673, 2.6418249598285514e-83) 
 low: (0.15084728146352522, 1.6538417536261343e-06)


In [135]:
# make the dataset 
y = 10 + 1*x1 + 2*x2 + .05*x4  + np.random.standard_normal(1000) * 0.2

df = pd.DataFrame({'y':y, 'x1':x1 , 'x2':x2, 'x3_nn': x3_nn,
                   'x3_high': x3_high, 'x3_med':x3_med, 'x3_low':x3_low ,
                   'x5': x5 } )
                  
print(df.describe())
df.to_stata('data/og_xy.dta', write_index=False) 

                 y           x1           x2        x3_nn      x3_high  \
count  1000.000000  1000.000000  1000.000000  1000.000000  1000.000000   
mean     16.410458     5.000000     0.058070   100.326000   100.338578   
std       3.100842     2.891085     0.689692    20.745701    20.795577   
min      10.181842     0.000000    -0.999987    52.000000    51.465077   
25%      13.978295     2.500000    -0.615909    85.000000    85.212311   
50%      16.370703     5.000000     0.109760   100.000000   100.937288   
75%      18.944874     7.500000     0.724484   116.000000   115.998077   
max      24.048262    10.000000     0.999999   148.000000   146.986390   

            x3_med       x3_low           x5  
count  1000.000000  1000.000000  1000.000000  
mean    100.139694    98.770663    74.524000  
std      25.619513   102.304104    14.047375  
min      23.285782  -255.068118    50.000000  
25%      82.064478    28.769345    62.000000  
50%     100.001875   103.530740    75.000000  
75% 

do gp stuff 

In [136]:
#rbf = ConstantKernel(1.0) * RBF(length_scale=1.0)  #+ WhiteKernel(1)
rbf = RBF(length_scale=1.0)
gpr = GaussianProcessRegressor(kernel=rbf )

#df = (df - df.mean()) # /df.std() 
#print(df.describe())
      
df2 = pd.DataFrame()


for x in df.columns:
    
    # variable with respect to which we are whitening 
    y = df[x]
    #print(y.mean())
    
    #X = np.asarray(df[[i for i in df.columns if i not in x ]]).reshape(-1,1)
    #X = df[[i for i in df.columns if i not in x ]]
    X = df[['x5']]
    
    gpr = GaussianProcessRegressor(kernel=rbf, alpha=.01 ).fit(X,y)
    print(f"for var:{x} - {gpr.kernel_}")
    
    y_hat = gpr.predict(X)
    #plt.plot(x ,y_hat)
    
    res = y - y_hat 
    df2[x] = res     

df2.describe()

#normalized_df=(df2-df2.mean())/df2.std() 
#normalized_df.to_stata('data/xy_wh.dta', write_index=False)
df2.to_stata('data/xy_wh_corr.dta', write_index=False)

for var:y - RBF(length_scale=1.16)
for var:x1 - RBF(length_scale=0.936)
for var:x2 - RBF(length_scale=1.09)
for var:x3_nn - RBF(length_scale=1.28)
for var:x3_high - RBF(length_scale=1.28)
for var:x3_med - RBF(length_scale=1.15)
for var:x3_low - RBF(length_scale=0.841)
for var:x5 - RBF(length_scale=33.4)


# 